# Preeliminaries

### Rpy2 package

In [1]:
%load_ext rpy2.ipython

### Importing R packages

In [2]:
%R library(latentcor)
%R library(glmnet)
%R library(survival)
%R library(dplyr)

R[write to console]: Loading required package: Matrix

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




'dplyr','survival','glmnet',...,'datasets','methods','base'


### Importing Python libraries

In [3]:
import pandas as pd
import warnings
import numpy as np
from timeit import default_timer as timer
import math


#For parallel computing
import multiprocessing
from joblib import Parallel, delayed
#Number of cores
num_cores = multiprocessing.cpu_count()
jobs=num_cores-1


import matplotlib.pyplot as plt
%matplotlib inline

#knockpy (knockoffs)
import knockpy
from knockpy.knockoff_filter import KnockoffFilter
from knockpy.knockoff_stats import data_dependent_threshhold

#GGlasso (graphical Lasso)
from gglasso.problem import glasso_problem

In [4]:
#Import the package rpy2
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

# import R's packages
base = importr('base')
glmnet = importr('glmnet')
dplyr = importr('dplyr')
survival = importr('survival')

### Auxiliary functions

#### Python functions

In [5]:
#Function to make selections
def make_selections(W, fdr):
    """" Calculate data dependent threshhold and selections
    returns a np.ndarray
    
    Parameters 
    ---------- 
    W : np.ndarray 
    fdr : float
    """  
    
    threshold = data_dependent_threshhold(W=W, fdr=fdr)
    selected_flags = (W >= threshold).astype("float32")
    return selected_flags


def lasso_glmnet_lambda_min(x):
  """" Find the tuning lambda using the R package glmnet 
  
  Returns a robjects.vectors.FloatVector
  
  Parameters 
  ---------- 
  x : pandas.DataFrame  
  """    
    
  #Convertion of the pandas dataframe to a R dataframe  
  sim = x
  with localconverter(robjects.default_converter + pandas2ri.converter):
    r_sim = robjects.conversion.py2rpy(sim)
  robjects.globalenv["r_sim"] = r_sim
  
  #Loading R libraries  
  base = importr('base')
  glmnet = importr('glmnet')
  dplyr = importr('dplyr')
  survival = importr('survival')

  #Fitting the Cox’s proportional hazards model employing glmnet
  robjects.r(''' 
        X <- r_sim %>% select(-c("Pat_Died", "Pat_Overall_Survival_Months"))
        X_matrix <- as.matrix(X)
        y <- r_sim %>% select(c("Pat_Died", "Pat_Overall_Survival_Months"))
        y_surv <- Surv(y$Pat_Overall_Survival_Months,y$Pat_Died)
        cvfit <- cv.glmnet(X_matrix, y_surv, alpha=1, family = "cox", type.measure = "C", nfolds = 5, standardize = TRUE)
        lambda_min_r <- as.numeric(cvfit$lambda.min)
        ''')
  #Tuning lambda
  lambda_min = robjects.globalenv['lambda_min_r']  
  
  return lambda_min  


#### R functions

In [6]:
%%R

#Transformation to get the corresponding latent normal variable for each original ordinal variable
transformation_to_get_latent_normal_variable <- function(original_variable){
     
    
  var_table <- table(original_variable) # Table object to obtain the levels of the ordinal variable
  n_levels <- dim(var_table) #Levels of the original ordinal variable
  n <- sum(var_table) #Number of observations
  
  if (n_levels > 3) 
        stop("Ordinal variables with more than 3 levels must be considered continuous, 
              because latentcor estimation is limited to binary or ternary ordinal variables")  
    
    
  #Condition when the original ordinal variable has two levels  
  if(n_levels==2){
    Xj_1 <- as.numeric(var_table[2])/n 
    fCj_1 <- qnorm(1-Xj_1) #Transformed cutoffs (latent normal level)
    nj_1 <- (abs(fCj_1-(-3))/2)+(-3) #Middle point of the interval ( fCj_0=-inf, fCj_1), where -inf is replace by -3  
    nj_2 <- (abs(3-fCj_1)/2) + fCj_1 #Middle point of the interval ( fCj_1, fCj_2=inf ), where inf is replace by 3  
 
    #The latent Gaussian transformed value is chosen as the middle point of the interval ( fCj_i, fCj_{i+1} ). 
    #The extreme points -inf and inf were replace for -3 and 3, respectively.
    transformed_variable <- original_variable
    transformed_variable[original_variable==as.numeric(names(var_table)[1])]= nj_1
    transformed_variable[original_variable==as.numeric(names(var_table)[2])]= nj_2
  }
  #Condition when the original ordinal variable has three levels    
  else{
    Xj_1 <- as.numeric(var_table[2]+var_table[3])/n  
    fCj_1 <- qnorm(1-Xj_1) #Transformed cutoffs (latent normal level)
    nj_1 <- (abs(fCj_1-(-3))/2)+(-3) #Middle point of the interval ( fCj_0=-inf, fCj_1 ), where -inf is replace by -3  
    Xj_2 <- as.numeric(var_table[3])/n  
    fCj_2 <- qnorm(1-Xj_2 ) #Transformed cutoffs (latent normal level)
    nj_2 <- (abs(fCj_2-fCj_1)/2)+(fCj_1) #Middle point of the interval ( fCj_1, fCj_2 )  
    nj_3 <- (abs(3-fCj_2)/2) + fCj_2 #Middle point of the interval ( fCj_2, fCj_3=inf ), where inf is replace by 3.  
 
  
    #The latent Gaussian transformed value is chosen as the middle point of the interval ( fCj_i,fCj_{i+1} ). 
    #The extreme points -inf and inf were replace for -3 and 3, respectively.
    transformed_variable <- original_variable
    transformed_variable[original_variable==as.numeric(names(var_table)[1])] = nj_1
    transformed_variable[original_variable==as.numeric(names(var_table)[2])] = nj_2
    transformed_variable[original_variable==as.numeric(names(var_table)[3])] = nj_3
  } 

return(transformed_variable)
}


#Transformation to obtain the orginal ordinal variable from the latent Gaussian knockoff
transformation_to_get_original_variable <- function(original_variable, normal_variable){

  var_table <- table(original_variable) #Table object to obtain the levels of the original ordinal variable
  n_levels <- dim(var_table) #Levels of the original ordinal variable
  n <- sum(var_table) #Number of observations 
  original_varible_unique_values <- sort(unique(original_variable)) #Unique values of the original ordinal variable
  ordinal_variable <- rep(0,length(normal_variable))
    
  
  if (n_levels > 3) 
      stop("Ordinal variables with more than 3 levels must be considered continuous, 
              because latentcor estimation is limited to binary or ternary ordinal variables") 
    
  #Condition when the original ordinal variable has two levels  
  if(n_levels==2){
    Xj_1 <- as.numeric(var_table[2])/n
    fCj_1 <- qnorm(1-Xj_1)  #Transformed cutoffs (latent normal level)
      
    #Ordinal_variable assignation depending on the transformed cutoffs
    ordinal_variable[normal_variable<=fCj_1] <- original_varible_unique_values[1]  
    ordinal_variable[normal_variable>fCj_1] <- original_varible_unique_values[2]  

  }
  #Condition when the original ordinal variable has three levels    
  else {
    Xj_1 <- as.numeric(var_table[2]+var_table[3])/n  
    fCj_1 <- qnorm(1-Xj_1) #Transformed cutoffs (latent normal level)
    Xj_2 <- as.numeric(var_table[3])/n
    fCj_2 <- qnorm(1-Xj_2 ) #Transformed cutoffs (latent normal level) 
      
    #Ordinal_variable assignation depending on the trnsformed cutoffs  
    ordinal_variable[normal_variable<=fCj_1] <- original_varible_unique_values[1]  
    ordinal_variable[normal_variable>fCj_1 & normal_variable<=fCj_2] <- original_varible_unique_values[2]  
    ordinal_variable[normal_variable>fCj_2] <- original_varible_unique_values[3]  
   
    }
return(ordinal_variable)    
}    

In [9]:
%%R

#Function to identify if the variable is continuous(con) or ordinal (ord)
column_type_identification <- function(col){ 
  if(length(unique(col)) < 4)
  {type<-"ord"}
  else
  {type<-"con"} 
  return(type)
}    


#Function to identify if the variable is continuous(con), binary (bin) or ternary (ter)
latentcor_type_identification <- function(col) { 
  if(length(unique(col)) == 2)
  {type<-"bin"}
  else if (length(unique(col)) == 3)
  {type<-"ter"}
  else
  {type<-"con"} 
  return(type) 
}

### Parameter configuration

In [10]:
#Initial time
tii = timer()


n_cv = 5  #Cross validation folds
M = 200 #Runs for stabilizing the lasso against CV fold assignation

# Loading the Lung cancer dataset

In [11]:
#Importing the csv file (307 rows and 5005 columns)
Lung_data_complete = pd.read_csv("Patients_final_5003.csv")

#Encoding for binary variables
Lung_data_complete['Pat_Gender'] = np.where(Lung_data_complete['Pat_Gender']=="M",0,1)
Lung_data_complete['Pat_Stage_red'] = np.where(Lung_data_complete['Pat_Stage_red']=="I_II",0,1)


#Removing NA's and nan's from the dataframe 
Lung_data_complete = Lung_data_complete.dropna(axis=0, how='any')
#15 observations are removed (less than 5%) 


#4 observations that have a survival time equal to zero, which causes a problem when fitting the Cox-lasso model.
#Therefore, these values are modified to have a survival time of 1 day (1/30)
Lung_data_complete.loc[Lung_data_complete.Pat_Overall_Survival_Months == 0, "Pat_Overall_Survival_Months"] = 0.033 


### Previous step that filters the most variable genes in terms of their variance

In [12]:
#Notes: 
#1) In the Lung_data data frame, the gene-expression column order is from the most variable to the less variable, 
#in terms of their variance. 
#2) The Lung_data contains the most 5000 expressed genes of a total of 20,356 genes of the
# genomic dataset from the research of Rousseaux et al.(2013) located in the Lung Cancer Explorer (LCE) database 
# http://lce.biohpc.swmed.edu/.

#Number of most expressed gene selected: 
gen_p = 289

#Reduce dataframe with the most expressed genes given by gen_p 
#(there are 3 clinical variables and 2 variables associated to survival time and event type)
Lung_data = Lung_data_complete.iloc[:,0: gen_p + 3 +2]

#From Python to R
%R -i Lung_data


### Computing the censoring rate

In [13]:
%%R

Lung_data 
Censoring <- round(100*as.numeric(table(Lung_data$Pat_Died)[1]/length(Lung_data$Pat_Died)),2)
print(Censoring)

[1] 31.85


# Applying the Cox's PH model with Lasso penalization

### Stabilizing the lasso against CV (Roberts and Nowak, 2014)
Fitting the penalized regression model  M times  to get M different values of the tuning parameter lambda.

In [14]:
ti = timer() #Initial time

ls_Lung_data = list(range(M)) #List of pandas dataframes needed for the parallel processing
for i in range(M):
  ls_Lung_data[i] = Lung_data


#Parallel code with Joblib
ls_lambdas = Parallel(n_jobs=jobs)(delayed(lasso_glmnet_lambda_min)(x) for x in ls_Lung_data)

time_parallel_computing = timer() - ti #Final time of parallel computing
print('Time (min) taken to run the parallel computing',round(time_parallel_computing/60,4))


#Transforming the list to a numpy array
lambdas = np.array(ls_lambdas)

#From python to R
%R -i lambdas 

Time (min) taken to run the parallel computing 3.1005


### Creating matrices: X and y_surv

In [15]:
#4 observations that have a survival time equal to zero, which causes a problem when fitting the Cox-lasso model.
#Therefore, these values are modified to have a survival time of 1 day (1/30)
%R Lung_data$Pat_Overall_Survival_Months[Lung_data$Pat_Overall_Survival_Months==0] <- 0.033

#Matrix generation
%R X <- Lung_data %>% select(-c("Pat_Died", "Pat_Overall_Survival_Months"))
%R X_matrix <- as.matrix(X)
%R y <- Lung_data %>% select(c("Pat_Died", "Pat_Overall_Survival_Months"))
%R y_surv <- Surv(y$Pat_Overall_Survival_Months,y$Pat_Died)

array([[2.21e+02, 0.00e+00],
       [1.10e+01, 1.00e+00],
       [5.70e+01, 1.00e+00],
       [4.30e+01, 1.00e+00],
       [9.50e+01, 1.00e+00],
       [3.40e+01, 1.00e+00],
       [1.20e+01, 0.00e+00],
       [3.30e+01, 1.00e+00],
       [2.40e+01, 0.00e+00],
       [2.00e+02, 0.00e+00],
       [8.00e+01, 1.00e+00],
       [1.21e+02, 0.00e+00],
       [2.10e+01, 1.00e+00],
       [8.60e+01, 1.00e+00],
       [1.20e+01, 1.00e+00],
       [1.95e+02, 1.00e+00],
       [6.80e+01, 0.00e+00],
       [5.20e+01, 1.00e+00],
       [7.00e+01, 1.00e+00],
       [1.15e+02, 0.00e+00],
       [7.20e+01, 1.00e+00],
       [1.18e+02, 0.00e+00],
       [1.04e+02, 1.00e+00],
       [5.00e+00, 1.00e+00],
       [2.18e+02, 0.00e+00],
       [8.70e+01, 0.00e+00],
       [8.50e+01, 0.00e+00],
       [5.10e+01, 0.00e+00],
       [1.34e+02, 0.00e+00],
       [3.60e+01, 1.00e+00],
       [5.40e+01, 0.00e+00],
       [3.40e+01, 1.00e+00],
       [2.28e+02, 1.00e+00],
       [2.00e+01, 1.00e+00],
       [5.80e+

### Fitting the Cox’s proportional hazards model with Lasso (original predictors X)

In [16]:
%%R

#The 50-percentile corresponds to the usual Cox-lasso
lambda50 <- as.numeric(quantile(lambdas,probs=0.5)) 

#Fitting the final model with the tunned lambda
fit <- glmnet(X_matrix,y_surv,alpha = 1, lambda = lambda50, family = "cox", standardize = TRUE)

fit_coef <- coef(fit)

#Showing the variables selected (coefficients different to zero)
print(fit_coef[fit_coef[,1]!=0,])

      Pat_Age Pat_Stage_red         51297          5055          6696 
 1.937963e-02  5.087666e-01 -2.199087e-03 -8.438380e-06  3.046971e-02 
         1428         28781          6590        283120          3495 
-1.453308e-01 -2.690735e-02 -1.480740e-01  3.333601e-02 -3.848062e-02 
         9787 
 2.795149e-01 


# Applying the Model-X knockoff methodology

## 1) knockoff construction using the LGCK procedure


### Data preparation 

In [17]:
#Design matrix X
data_X = Lung_data.drop(['Pat_Died','Pat_Overall_Survival_Months'], axis=1)

#Necessary elements  to calculate the truncated empirical cumulative distribution funcion (ECDF) estimator
n = data_X.shape[0] #Number of observations
p = data_X.shape[1] #Number of covariates
delta_n = 1/( (4*n**(1/4))*math.sqrt(math.pi*math.log(n)) )

#From Python to R
%R -i data_X
%R -i p
%R -i delta_n

### Step 1 of the LGCK procedure: Estimation of the latent correlation matrix.

In [18]:
# Column identification
%R column_type <- apply(data_X, MARGIN = 2, FUN = column_type_identification)
%R latentcor_type <- apply(data_X, MARGIN = 2, FUN = latentcor_type_identification)

#Latencor estimation
%R latentcor_hat <- latentcor(data_X, type=latentcor_type, method="original")$R


array([[ 1.        , -0.11320615, -0.11700009, ..., -0.08648349,
         0.03196515, -0.03861371],
       [-0.11320615,  1.        ,  0.02135272, ...,  0.13807362,
        -0.10099819,  0.11955142],
       [-0.11700009,  0.02135272,  1.        , ..., -0.21852877,
         0.36934779, -0.37179449],
       ...,
       [-0.08648349,  0.13807362, -0.21852877, ...,  1.        ,
        -0.19845127,  0.33284737],
       [ 0.03196515, -0.10099819,  0.36934779, ..., -0.19845127,
         1.        , -0.2492608 ],
       [-0.03861371,  0.11955142, -0.37179449, ...,  0.33284737,
        -0.2492608 ,  1.        ]])

### Step 2 of the LGCK procedure: Estimation of the precision matrix of the latent correlation matrix.

In [19]:
#From Python to R
%R -o latentcor_hat 

#Instantiate the  glasso_problem
P = glasso_problem(S=latentcor_hat, N=n, reg_params = {'lambda1': 0.05}, latent = False, do_scaling = False)

# Next, do model selection by solving the problem on a range of lambda values.
lambda1_range = np.logspace(1, -5, 30)
modelselect_params = {'lambda1_range': lambda1_range}
P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 0.1)

#Precision and Sigma matrices
sol = P.solution.precision_
Sigma_hat = np.linalg.inv(sol)

ADMM terminated after 23 iterations with status: optimal.
ADMM terminated after 12 iterations with status: optimal.
ADMM terminated after 29 iterations with status: optimal.
ADMM terminated after 12 iterations with status: optimal.
ADMM terminated after 12 iterations with status: optimal.
ADMM terminated after 25 iterations with status: optimal.
ADMM terminated after 172 iterations with status: optimal.
ADMM terminated after 12 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 9 iterations with status: optimal.
ADMM terminated after 254 iterations with status: optimal.
ADMM terminated after 226 iterations with status: optimal.
ADMM terminated after 123 iterations with status: optimal.
ADMM terminated after 135 iterations with status: optimal.
ADMM terminated after 135 iterations with status: optimal.
ADMM terminated after 153 iterations with status: optimal.
ADMM terminated after 182 iterations with status: optimal.
ADMM te

### Step 3 of the LGCK procedure: Nonparametric transformation strategy to obtain marginal normality.

In [20]:
%%R

#Matrices for ECDF and X_norm_hat (contains the sample Gaussian Knockoffs)
X_ecdf <- data_X 
X_norm_hat <- data_X

#Empirical cumulative distribution function
for(i in 1:p) {  
    if(column_type[i]=="con"){ 
        X_ecdf[,i] <- as.vector(ecdf(data_X[,i])(data_X[,i])) 
    }
}

#Truncation for computing the Winsorized ECDF
for(i in 1:p) {   
    if(column_type[i]=="con"){ 
        X_ecdf[,i][ X_ecdf[,i] < delta_n] <- delta_n 
    }
}
for(i in 1:p) {   
    if(column_type[i]=="con"){ 
        X_ecdf[,i][ X_ecdf[,i] > (1-delta_n)] <- 1-delta_n 
    }
}   

#Getting normal margins for continuous variables
for(i in 1:p) {   
    if(column_type[i]=="con"){ 
        X_norm_hat[,i] <- as.vector(qnorm( X_ecdf[,i] ) )
    }
}
  
#Transformation to get the corresponding latent normal variable for each ordinal variable
for(i in 1:p) {   
    if(column_type[i]=="ord"){ 
        X_norm_hat[,i] <- transformation_to_get_latent_normal_variable(data_X[,i])
    }
}  

### Step 4 of the LGCK procedure. Sampling Gaussian knockoffs using the MRV approach

In [21]:
#From R to Python
%R -o X_norm_hat

#set seed
np.random.seed(1)

#Instantiating an object of the class GaussianSampler for sampling 
#Gaussian knockoffs using the estimated Sigma_hat and the method mvr
Gaussian_sampler_hat = knockpy.knockoffs.GaussianSampler(X_norm_hat.to_numpy(), mu=None,
                                                           Sigma=Sigma_hat,
                                                           method='mvr', verbose=False)
#Samplign the Gaussian Knockoffs
Xk_norm_hat = Gaussian_sampler_hat.sample_knockoffs() 


#Creating a dataframes from the array that contains the Gaussian Knockoffs
df_Xk_norm_hat = pd.DataFrame(Xk_norm_hat)

#From Python to R
%R -i df_Xk_norm_hat


    

### Step 5 of the LGCK procedure. Reversing transformation to obtain the non-Gaussian Knockoffs.

In [22]:
#Dataframe to save the transformations
%R df_Xk_hat <- df_Xk_norm_hat

#Transformation to obtain the original continuous variable from the Gaussian knockoff  
%R for(i in 1:p) {   if(column_type[i]=="con"){ df_Xk_hat[,i] <- as.vector(quantile(data_X[,i], probs=pnorm(df_Xk_norm_hat[,i]), type=8)) }}
#Transformation to obtain the orginal ordinal variable from the latent Gaussian knockoff
%R for(i in 1:p) {   if(column_type[i]=="ord"){ df_Xk_hat[,i] <- transformation_to_get_original_variable(original_variable=data_X[,i], normal_variable=df_Xk_norm_hat[,i])}}

#From R to Python
%R -o df_Xk_hat
df_Xk_hat.reset_index(drop=True, inplace=True)

#Creating the names for the variables in Xk
columns_names = list(data_X.columns)
k_columns_names = ['K_'+ str(name) for name in columns_names]
df_Xk_hat.columns = k_columns_names

#Creating the dataset (X, Xk) (original variables + knockoffs)
data_X.reset_index(drop=True, inplace=True)
data_X_Xk = pd.concat([data_X, df_Xk_hat], axis=1)
  

## 2) Knockoff statistic estimation 

### Creating the Lung_data_Xk dataframe (Lung_data + Knockoffs Xk)¶


In [23]:
#Selecting the survival time and event indicator
y_surv = Lung_data.filter(['Pat_Died','Pat_Overall_Survival_Months'], axis=1)
y_surv.reset_index(drop=True, inplace=True)

#New dataset with the knockoffs
Lung_data_Xk = pd.concat([y_surv, data_X_Xk], axis=1)
  

### Stabilizing the lasso against CV (Roberts and Nowak, 2014)
Fitting the penalized regression model  M times to get M different values of the tuning parameter lambda.

In [24]:
ti = timer() #Initial time

ls_Lung_data_Xk = list(range(M)) #List of pandas data frames needed for the parallel processing
for i in range(M):
  ls_Lung_data_Xk[i] = Lung_data_Xk


#Parallel code with Joblib
ls_lambdas_Xk = Parallel(n_jobs=jobs)(delayed(lasso_glmnet_lambda_min)(x) for x in ls_Lung_data_Xk)


time_parallel_computing = timer() - ti #Final time of parallel computing
print('Time (min) taken to run the parallel computing',round(time_parallel_computing/60,4))


#Transforming the list to a numpy array
lambdas_Xk = np.array(ls_lambdas_Xk)

#From Python to R
%R -i lambdas_Xk 


Time (min) taken to run the parallel computing 1.2165


### Creating the matrix (X,Xk) (Original variables X + Knockoffs Xk)

In [25]:
#From Python to R
%R -i Lung_data_Xk

#Matrix generation
%R X_Xk <- Lung_data_Xk %>% select(-c("Pat_Died", "Pat_Overall_Survival_Months"))
%R X_Xk_matrix <- as.matrix(X_Xk)


array([[ 1.        , 60.        ,  0.        , ...,  0.26363052,
        -0.88803707, -0.46701719],
       [ 0.        , 52.        ,  1.        , ...,  1.6066931 ,
        -0.3855968 , -0.4967244 ],
       [ 0.        , 66.        ,  0.        , ...,  0.89208975,
        -0.76690606,  1.32898096],
       ...,
       [ 0.        , 51.        ,  0.        , ..., -1.21061369,
         1.87386177, -0.80302497],
       [ 0.        , 60.        ,  1.        , ..., -0.92949962,
        -0.16210037, -0.25827246],
       [ 0.        , 66.        ,  0.        , ...,  0.95160549,
        -0.73107787, -0.56504696]])

### Fitting the Cox’s proportional hazards model with Lasso (Original variables X + Knockoffs Xk) 

In [26]:
%%R

#The 50-percentile corresponds to the usual Cox-lasso
lambda50_Xk <- as.numeric(quantile(lambdas_Xk,probs=0.5))

#Fitting the final model with the tunned lambda
fit_LCGK <- glmnet(X_Xk_matrix, y_surv, alpha = 1, lambda =lambda50_Xk, family = "cox", standardize = TRUE)

fit_LCGK_coef <- coef(fit_LCGK)

###  Computing the variable importance statistic (Z) and the knockoff feature statistic (Wj)

In [27]:
#Transforming the sparse matrix to a vector
%R fit_LCGK_coef_vec <- as.vector(fit_LCGK_coef)

#From R to Python
%R -o fit_LCGK_coef_vec

#Variable importance statistic
Z = fit_LCGK_coef_vec

#Knockoff statistic (Wj)
pair_W = np.abs(Z[0:p]) - np.abs(Z[p:])


## 3) Data-dependent threshold calculation 

In [28]:
#Target false discovery rate
FDR=0.25

#Data-dependent threshold calculation
threshold = data_dependent_threshhold(W=pair_W, fdr= FDR)
print("Threshold for knockoffs ")
print(threshold)
rejections = make_selections(W=pair_W, fdr= FDR)

#Number of selections (rejections of the null hypothesis of Y independent of Xj given X-j)
Number_Rejections_knockoff_hat = rejections.sum()
print("Number of non-zero knockoff coefficients: {}".format(Number_Rejections_knockoff_hat))


Threshold for knockoffs 
0.004189289911246587
Number of non-zero knockoff coefficients: 8.0


# Results of selected features

### Selected variables (LGCK-LCD procedure)

In [29]:
data_X.columns.values[rejections==1]

array(['Pat_Age', 'Pat_Stage_red', '6696', '1428', '6590', '283120',
       '3495', '9787'], dtype=object)

### Selected variables (Cox-lasso)

In [30]:
%%R
#Showing the variables selected (coefficients different to zero)
print(fit_coef[fit_coef[,1]!=0,])
print(length(fit_coef[fit_coef[,1]!=0,]))

      Pat_Age Pat_Stage_red         51297          5055          6696 
 1.937963e-02  5.087666e-01 -2.199087e-03 -8.438380e-06  3.046971e-02 
         1428         28781          6590        283120          3495 
-1.453308e-01 -2.690735e-02 -1.480740e-01  3.333601e-02 -3.848062e-02 
         9787 
 2.795149e-01 
[1] 11


In [31]:
print('Number of features (p):',p) #Number of features of the initial dataset: lung_data

tff = timer()
print('Time (min) taken to run all is:',round((tff-tii)/60,4)) #Computing time

Number of features (p): 292
Time (min) taken to run all is: 5.7812


R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to cons